In [1]:
#连接的usb端口
%serialconnect to --port=/dev/user_voice --baud=115200

Connecting to --port=/dev/user_voice --baud=115200 
Ready.


In [2]:
from fpioa_manager import *
from Maix import I2S, GPIO
import audio

In [3]:
AUDIO_PA_EN_PIN = 32  # 功放使能信号引脚

# 功放使能
if AUDIO_PA_EN_PIN:
    fm.register(AUDIO_PA_EN_PIN, fm.fpioa.GPIO1, force=True)
    audio_en = GPIO(GPIO.GPIO1, GPIO.OUT)
    audio_en.value(0)

# register i2s(i2s0) pin 数字扬声器引脚
fm.register(34, fm.fpioa.I2S1_OUT_D1, force=True)
fm.register(35, fm.fpioa.I2S1_SCLK, force=True)
fm.register(33, fm.fpioa.I2S1_WS, force=True)

# init i2s(i2s0)初始化
wav_dev = I2S(I2S.DEVICE_1)
wav_dev.channel_config(wav_dev.CHANNEL_1, wav_dev.TRANSMITTER, align_mode=I2S.RIGHT_JUSTIFYING_MODE)

In [4]:
# init audio
player = audio.Audio(path="/flash/haode.wav")  # 引用播放文件
player.volume(40)  # 音量

# read audio info
wav_info = player.play_process(wav_dev)  # 读取文件
print("wav file head information: ", wav_info)

wav_dev.channel_config(wav_dev.CHANNEL_1, wav_dev.TRANSMITTER, align_mode=I2S.RIGHT_JUSTIFYING_MODE)  # 使用该设备播放
wav_dev.set_sample_rate(wav_info[1])

audio_en.value(1)  # 使能功放
while True:
    ret1 = player.play()  # 播放音频
    if ret1 == 0:
        # print("end")
        break
player.finish()  # 结束播放
audio_en.value(0)  # 关闭功放

[MAIXPY]: result = 0
[MAIXPY]: numchannels = 1
[MAIXPY]: samplerate = 16000
[MAIXPY]: byterate = 32000
[MAIXPY]: blockalign = 2
[MAIXPY]: bitspersample = 16
[MAIXPY]: datasize = 17280
wav file head information:  [1, 16000, 32000, 2, 16, 17280]
